# Sentinel-1

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio

import asf_search as asf
from hyp3_sdk import HyP3

from vegmapper import s1
from vegmapper import pathurl

## User Inputs

In [ ]:
# Site name
sitename = 'ucayali'

# Project directory (local path or cloud bucket URL)
proj_dir = sitename

# Convert to a ProjDir object, which works like a pathlib.Path object
proj_dir = pathurl.ProjDir(proj_dir)

# AOI file
aoifile = f'{proj_dir}/ucayali_boundary.geojson'

# Start and end dates of interest
start_date = '2021-01-01'
end_date = '2021-12-31'

1. [Granule Search](#1.-Granule-Search)
2. [Radiometric Terrain Correction (RTC)](#2.-Radiometric-Terrain-Correction-(RTC))
3. [Post-Processing](#3.-Post-Processing)
    - [Get RTC products](#Get-RTC-products)
    - [Build VRTs](#Build-VRTs)
    - [Calculate temporal mean](#Calculate-temporal-mean)
    - [Remove edges](#Remove-edges)

In [ ]:
# Plot AOI
gdf_aoi = gpd.read_file(aoifile)
gdf_aoi.plot(figsize=(10, 10))

## 1. Granule Search

ASF DAAC uses *granules* and *scenes* interchangeably to refer to a Sentinel-1 product temporally and geographically, whereas *frames* are used to refer to the geolocation only for a Sentinel-1 product. The naming convention for a Sentinel-1 granule can be found [here](https://asf.alaska.edu/data-sets/sar-data-sets/sentinel-1/sentinel-1-data-and-imagery/). Each *frame* can be uniquely identified by a pair of *path* and *frame* numbers. In this section, we will search for Sentinel-1 granules that intersect with AOI and were acquired between the start and end dates.

### `s1.search_granules`

```
s1.search_granules(sitename, aoifile, start_date, end_date, skim=True, **search_opts)
```

Paremeters:

|Paremeters||Description||Required||Default|
|----||----||----||----|
|`sitename`||Site name||Yes|||
|`aoifile`||AOI file in vector-based spatial data format (shapefile, GeoJSON, ...)||Yes|||
|`start_date`||Start date (YYYY-MM-DD)||Yes|||
|`end_date`||End date (YYYY-MM-DD)||Yes|||
|`skim`||Skim the search results so only the frames that just cover the AOI are retained||No||True|
|`search_opts`||Search options for ASF Python module (asf_search). See [here](https://docs.asf.alaska.edu/asf_search/searching/).||No||True|

Returns:

|Returns||Description|
|----||----|
|`gdf_granules`||A GeoDataFrame containing all searched granules along with their detailed properties|
|`gdf_frames`||A GeoDataFrame of `gdf_granules` grouped by frames.|

In [ ]:
# Here we search for Sentinel-1 Ground Range Detected (GRD) High Resolution (HD) products acquired with Interferometric Wide (IW) beam mode and both VV and VH polarizations.
search_opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'processingLevel': asf.PRODUCT_TYPE.GRD_HD,
    'beamMode': asf.BEAMMODE.IW,
    'polarization': asf.POLARIZATION.VV_VH,
}
gdf_granules, gdf_frames = s1.search_granules(sitename, aoifile, start_date, end_date, skim=True, **search_opts)

In [ ]:
gdf_granules

In [ ]:
gdf_frames

In [ ]:
# Plot search results
ax = gdf_aoi.plot(figsize=(10, 10))
gdf_granules.boundary.plot(ax=ax, color='red')

## 2. Radiometric Terrain Correction (RTC)

For the initial processing of the Sentinel-1 granules, we make use of ASF's HyP3 API. Information about the specifics of this processing can be found in the [HyP3 documentation](https://hyp3-docs.asf.alaska.edu/). `s1_submit_hyp3_jobs.py` will submit the granules chosen in the previous step to the HyP3 API for processing. The processed granules will be saved in the following directory structure, as .zip files.

### `s1.submit_hyp3_jobs`

```
s1.submit_hyp3_jobs(granules, hyp3=None, proj_dir=None, **rtc_opts)
```

|Parameters|Description|Required|Default|
|----|----|----|----|
|`granules`|GeoJSON file or gdf_granules|Yes||
|`hyp3`|HyP3 API.|No|None|
|`proj_dir`|Project directory|No|None|
|`rtc_opts`|RTC processing options|No|None|

In [ ]:
hyp3 = HyP3(prompt=True)

In [ ]:
# RTC processing options (https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_rtc_job)
rtc_opts = {
    'dem_matching': True,
    'dem_name': 'copernicus',
    'include_dem': False,
    'include_inc_map': True,
    'include_rgb': False,
    'include_scattering_area': False,
    'radiometry': 'gamma0',
    'resolution': 30,
    'scale': 'power',
    'speckle_filter': False,
}
batch = s1.submit_rtc_jobs(gdf_granules, proj_dir, hyp3, resubmit=False, **rtc_opts)

Check if all jobs in the batch are complete:

In [ ]:
batch.complete()

Download the completed RTC products:

In [ ]:
s1.download_files(batch, download_dir='hyp3_downloads')

Copy the downloaded files to `proj_dir`

In [ ]:
s1.copy_files(proj_dir, download_dir='hyp3_downloads')

## 3. Post-Processing

The post-processing of RTC products involves the following steps:
- [Get RTC products](#Get-RTC-products)
- [Build VRTs](#Build-VRTs)
- [Calculate temporal mean](#Calculate-temporal-mean)
- [Remove edges](#Remove-edges)

The dictionary `s1_proc` below contains the processing parameters and will be updated as we go through each step.

|Parameters|Description|Format|
|----|----|----|
|`s1_dir`|The Sentinel-1 directory under `proj_dir`|prod_dir / dirname|
|`start_date`|Start date|YYYY-MM-DD|
|`end_date`|End date|YYYY-MM-DD|
|`frames`|Frames to be processed|path_frame|

Note that `start_date` and `end_date` can be different than the ones you used when requesting RTC products. This gives you the flexibility if you want a different time period for your study.

In [ ]:
s1_proc = {
    's1_dir': proj_dir / 'Sentinel-1',
    'start_date': start_date,
    'end_date': end_date,
    'frames': None,
}

### Get RTC products

`s1.get_rtc_products`

```
s1.get_rtc_products(s1_proc)
```

Gets RTC products under `s1_dir` using `start_date`, `end_date`, and `frames` in `s1_proc` as search criteria.

In [ ]:
s1.get_rtc_products(s1_proc)

The paths of RTC products are now added to `s1_proc`

In [ ]:
s1_proc['frames']

### Build VRTs

`s1.build_vrt`

```
s1.build_vrt(s1_proc)
```

Builts VRTs to include all the RTC products in `s1_proc` for each frame. The data layers include:
- VV
- VH
- INC (incidence angle)
- LS (layover and shadow)

In [ ]:
s1.build_vrt(s1_proc)

In [ ]:
path_frame = list(s1_proc['frames'].keys())[0]
s1_proc['frames'][path_frame]['VV']

### Calculate temporal mean

`s1.calc_temporal_mean`

```
s1.calc_temporal_mean(s1_proc)
```

Calculates temporal mean for each VRT in `s1_proc`.

In [ ]:
s1.calc_temporal_mean(s1_proc)

In [ ]:
s1_proc['frames'][path_frame]

In [ ]:
with rasterio.open(s1_proc['frames'][path_frame]['VV']['mean'].path) as dset:
    VV = dset.read(1)
VV[VV == 0] = np.nan

In [ ]:
plt.rcParams['font.size'] = 18
plt.rcParams['figure.figsize'] = [8, 6]
plt.imshow(VV, vmin=0, vmax=0.5, cmap='Greys')

The artifacts at the left and right edges of Sentinel-1 VV/VH rasters are obvious and need to be removed.

In [ ]:
plt.imshow(VV[1750:2000, 0:400], vmin=0, vmax=0.5, cmap='Greys')

### Remove edges

`s1.remove_edges`

```
s1.remove_edges(s1_proc)
```

Remove left and right edges of the temporal mean rasters.

In [ ]:
s1.remove_edges(s1_proc)

In [ ]:
with rasterio.open(s1_proc['frames'][path_frame]['VV']['mean'].path) as dset:
    VV = dset.read(1)
VV[VV == 0] = np.nan
plt.imshow(VV[1750:2000, 0:400], vmin=0, vmax=0.5, cmap='Greys')